# Подготовка данных

## Настройка и загрузка датасета

In [14]:
from pathlib import Path
from vk_mod.data import load_dataset


SAMPLE_LIMIT:int = 40000
datasets = [
    Path('../data/preprocessed/image_only.csv'),
    Path('../data/preprocessed/text_only.csv'),
    Path('../data/preprocessed/text-and-image.csv')
]
images_dir = Path("..data/images")

df = load_dataset(datasets, sample_limit=SAMPLE_LIMIT)

## Небольшая статистика

In [15]:
has_text = df['text'].apply(lambda x: len(str(x).strip()) > 0)
has_image = df['image_name'].apply(lambda x: len(str(x).strip()) > 0)
print(f"""
      Статистика комбинаций данных:
      Только Текст: {len(df[has_text & ~has_image])}
      Только Картинка: {len(df[~has_text & has_image])} ЧС
      Только Текст+Картинка: {len(df[has_text & has_image])}
      Весь набор: {len(df)}
          """)
display(df.sample(10))


      Статистика комбинаций данных:
      Только Текст: 13333
      Только Картинка: 3553 ЧС
      Только Текст+Картинка: 8500
      Весь набор: 25386
          


,image_name,blocked,text
14566,,0,"скиньте пожалуйста перечень документов,необход..."
20344,92385.png,0,"Вы можете трахаться с этим дерьмом, вы можете ..."
11148,,0,я из дубоссар-хочу забрать собачку -черную с к...
16369,,0,люди не ведитесь за ними они несут нам вторую ...
6082,,0,https://aliexpress.ru/wholesale?catid=0initiat...
18039,07249.png,0,Обама бросает первый шаг в звездную игру
846,e621v32217.jpg,1,
13415,,0,"ничего удивительного,каждый день кабачки,вот с..."
13361,,1,"ското-база, пидор гнойный."
6745,,1,стадо баранов замбированые! ещё и агрессивных!


## Формирование выборок

In [16]:
from sklearn.model_selection import train_test_split
from vk_mod.const import SEED


train_df, val_df = train_test_split(df, test_size=0.2, random_state=SEED)
train_df, test_df = train_test_split(train_df, test_size=0.1, random_state=SEED, shuffle=False)

# Обучение модели

In [17]:
from vk_mod.models import toxic_classificator

model_path = Path("../pretrained_models/multi.keras")

model = toxic_classificator.train_model(
    train_df,
    val_df,
    images_dir,
    epochs=100,
    save_path=model_path
)

Epoch 1/100
    572/Unknown 433s 726ms/step - accuracy: 0.6929 - loss: 0.6076

c:\Users\ShapeCat\source\repos\ToxicPostDetection\.venv\lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


572/572 ━━━━━━━━━━━━━━━━━━━━ 537s 908ms/step - accuracy: 0.6929 - loss: 0.6076 - val_accuracy: 0.7475 - val_loss: 0.5512 - learning_rate: 1.0000e-04
Epoch 2/100
572/572 ━━━━━━━━━━━━━━━━━━━━ 488s 853ms/step - accuracy: 0.7524 - loss: 0.5342 - val_accuracy: 0.7499 - val_loss: 0.5234 - learning_rate: 1.0000e-04
Epoch 3/100
572/572 ━━━━━━━━━━━━━━━━━━━━ 562s 982ms/step - accuracy: 0.7689 - loss: 0.4819 - val_accuracy: 0.7686 - val_loss: 0.4870 - learning_rate: 1.0000e-04
Epoch 4/100
572/572 ━━━━━━━━━━━━━━━━━━━━ 498s 870ms/step - accuracy: 0.8283 - loss: 0.3930 - val_accuracy: 0.7725 - val_loss: 0.4911 - learning_rate: 1.0000e-04
Epoch 5/100
572/572 ━━━━━━━━━━━━━━━━━━━━ 497s 869ms/step - accuracy: 0.8674 - loss: 0.3181 - val_accuracy: 0.7708 - val_loss: 0.5129 - learning_rate: 5.0000e-05
Epoch 6/100
572/572 ━━━━━━━━━━━━━━━━━━━━ 468s 819ms/step - accuracy: 0.8812 - loss: 0.2869 - val_accuracy: 0.7739 - val_loss: 0.5241 - learning_rate: 2.5000e-05
Epoch 7/100
572/572 ━━━━━━━━━━━━━━━━━━━━ 468s 

# Оценка модели

## Оценка качества на разных комбинациях

In [18]:
toxic_classificator.evaluate_data_combinations(model, test_df, images_dir)

Данные: Только Текст
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 572ms/step - accuracy: 0.8732 - loss: 0.3402
Данные: Только Картинка
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 583ms/step - accuracy: 0.8564 - loss: 0.4215
Данные: Только Текст+Картинка
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 592ms/step - accuracy: 0.6386 - loss: 0.6589
Данные: Весь набор
64/64 ━━━━━━━━━━━━━━━━━━━━ 38s 593ms/step - accuracy: 0.7909 - loss: 0.4589


## Проверка сохранения

In [19]:
import keras


predict_after_training = toxic_classificator.predict_from_file(model, "это что за п*здец!?!!!!!", "e621v32986.jpg")
print(f"After training: {predict_after_training} -> {predict_after_training > 0.5}")

model = keras.models.load_model(model_path)
predict_after_reloading =  toxic_classificator.predict_from_file(model, "это что за п*здец!?!!!!!", "e621v32986.jpg")
print(f"After reloading: {predict_after_reloading} -> {predict_after_reloading > 0.5}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
After training: 0.1777285635471344 -> False
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
After reloading: 0.1777285635471344 -> False


## Проверка картинки по url

In [20]:
toxic_classificator.predict_from_url(model, "", "https://sun9-35.userapi.com/impg/doJp2md6Q18pWWtXHNgeMxgbcWQ2qSwHKG-BPA/VkvhrKl89Vc.jpg?size=1280x1280&quality=95&sign=25cb6e551f5b588be0d9c4fb52e2e767&type=album")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


0.873495

## Контент смешанного характера

### Положительный текст и отрицательная картинка

In [21]:
toxic_classificator.predict_from_url(model, "положительный текст", "https://sun9-35.userapi.com/impg/doJp2md6Q18pWWtXHNgeMxgbcWQ2qSwHKG-BPA/VkvhrKl89Vc.jpg?size=1280x1280&quality=95&sign=25cb6e551f5b588be0d9c4fb52e2e767&type=album")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


0.2476203

### Oтрицательная текст и положительная картинка

In [22]:
toxic_classificator.predict_from_url(model, "Что это за говно?!?!?!", "https://sun9-75.userapi.com/s/v1/if2/lP4YTYw5D4Ca1FYpQbgcZXhYQ-uI4Dv-Hg4TiYQ2l_9SDcqlpLqrqQQ5gCgjwmN0_sjj9YajX1BtDCdjP_hMVWmj.jpg?quality=96&as=32x24,48x36,72x54,108x81,160x120,240x180,360x270,480x360,540x405,640x480,720x540,1080x810,1280x960,1440x1080,2560x1920&from=bu&u=QCTk0zC8dlAqQg4g_uPSEn_dSMGvaqep7wUd5rG33y4&cs=807x605")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


0.5036142